In [28]:
import pandas as pd
import re
from datetime import datetime

# ExtractTransformLoad


## PlaneAccidents

In [29]:
df = pd.read_csv('./PI03-Analytics-main/AccidentesAviones.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)
df.info()
#nulls in this dataset are determined by a '?' string char

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5008 entries, 0 to 5007
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   fecha                   5008 non-null   object
 1   HORA declarada          5008 non-null   object
 2   Ruta                    5008 non-null   object
 3   OperadOR                5008 non-null   object
 4   flight_no               5008 non-null   object
 5   route                   5008 non-null   object
 6   ac_type                 5008 non-null   object
 7   registration            5008 non-null   object
 8   cn_ln                   5008 non-null   object
 9   all_aboard              5008 non-null   object
 10  PASAJEROS A BORDO       5008 non-null   object
 11  crew_aboard             5008 non-null   object
 12  cantidad de fallecidos  5008 non-null   object
 13  passenger_fatalities    5008 non-null   object
 14  crew_fatalities         5008 non-null   object
 15  grou

In [30]:
#let's rename the columns to a standard
dicc = {
'fecha': 'date',                   
'HORA declarada': 'declared_hour',        
'Ruta': 'route',            
'OperadOR': 'operator',              
'flight_no': 'flight_n',             
'route': 'route_type',                  
'ac_type': 'ac_type',                
'registration': 'registration',           
'cn_ln': 'cn_ln',                  
'all_aboard': 'aboard_total',            
'PASAJEROS A BORDO': 'aboard_pass',      
'crew_aboard': 'aboard_crew',            
'cantidad de fallecidos': 'deceased_aboard_total',
'passenger_fatalities': 'deceased_aboard_pass',  
'crew_fatalities':'deceased_aboard_crew',        
'ground': 'deceased_not_on_total',                 
'summary': 'summary'  
}
df = df.rename(columns=dicc)

In [31]:
#let's see the % of nulls
temp = df.replace('?',None)
temp.isnull().sum()/len(temp) *100

date                      0.000000
declared_hour            30.031949
route                     0.099840
operator                  0.199681
flight_n                 73.522364
route_type               15.215655
ac_type                   0.259585
registration              5.431310
cn_ln                    13.318690
aboard_total              0.339457
aboard_pass               4.412939
aboard_crew               4.373003
deceased_aboard_total     0.159744
deceased_aboard_pass      4.692492
deceased_aboard_crew      4.692492
deceased_not_on_total     0.878594
summary                   1.178115
dtype: float64

In [32]:
# here we can just drop the columns with way too many % of nulls, but because the flight_n is
#our best bet at getting JOIN data with Top100Fatalities.csv we'll leave it
df.drop('flight_n',axis=1,inplace=True)

    ### Date

In [33]:
#date: there seems to be nothing wrong with this column so I will just put it in date format
df.date = df.date.apply(lambda x: datetime.strptime(x, '%B %d, %Y'))

    ### Declared_hour

In [34]:
#declared_hour: this is in military hour so it needs to be 4 digits in length, so let's create a function that adapts to each type of length
df.declared_hour.apply(lambda x: len(x)).value_counts()

4    3289
1    1504
5     170
7      37
6       5
3       3
Name: declared_hour, dtype: int64

In [35]:
df[df.declared_hour.apply(lambda x: len(x)) == 4].declared_hour

0       1718
2       0630
4       1830
5       1030
6       0100
        ... 
5002    1705
5003    1835
5004    1800
5005    0800
5007    1500
Name: declared_hour, Length: 3289, dtype: object

In [36]:
df[df.declared_hour.apply(lambda x: len(x)) == 3].declared_hour

2140    900
3117    245
3794    175
Name: declared_hour, dtype: object

In [37]:
df[df.declared_hour.apply(lambda x: len(x)) == 5].declared_hour

132     10:00
156     14:48
160     11:30
170     08:00
171     10:45
        ...  
4380    02:30
4385    11:23
4531    05:30
4742    0500Z
5006    11:30
Name: declared_hour, Length: 170, dtype: object

In [38]:
df[df.declared_hour.apply(lambda x: len(x)) == 6].declared_hour

158     c 9:15
926     02:09Z
1120    03:50Z
1308    17:34Z
1340    01:00Z
Name: declared_hour, dtype: object

In [39]:
df.declared_hour.apply(lambda x: str(x)) #make sure everything is in string type

def clean_str(x):
    #I don't want null values to fill up the 00:00 time as it can change hour means/medians radically
    if x == '?':
        return x
    # let's find every non digit in the string and replace it with nothing
    x = re.sub(r'[^0-9]', '', x)
    # strip extra spaces    
    x = x.strip()
    # there's an outlier that would not be an hour
    if x == '175':
        x = x.ljust(4, '0')
    # fill up every <4 string with 0s
    x = x.zfill(4)
    return x
# apply it
df.declared_hour = df.declared_hour.apply(clean_str)

# now make it into the standard hh:mm format
def timeConvert(x):
    if x == '?':
        return x
    x = x[ :2] + ':' + x[2: ]
    pd.to_datetime(x, format='%H:%M')
    return x

df.declared_hour = df.declared_hour.apply(timeConvert)

    ### route

In [40]:
# there seem to be only 5 nulls here, so I can just leave them here
df[df.route == '?'].route

360     ?
465     ?
646     ?
999     ?
2895    ?
Name: route, dtype: object

    ### operator, route_type, ac_type, aboard_x, deceased_x, summary

In [41]:
#they have a really low number of nulls (the outlier here is route_type, but it still isn't enough to justify dropping it)
temp = df.replace('?',None)
temp = temp.iloc[:,3:]
temp.isnull().sum()/len(temp) *100

operator                  0.199681
route_type               15.215655
ac_type                   0.259585
registration              5.431310
cn_ln                    13.318690
aboard_total              0.339457
aboard_pass               4.412939
aboard_crew               4.373003
deceased_aboard_total     0.159744
deceased_aboard_pass      4.692492
deceased_aboard_crew      4.692492
deceased_not_on_total     0.878594
summary                   1.178115
dtype: float64

In [42]:
# I change null value from ? to real None type so each feature can have the correct data type in MySQL
df = df.replace('?',None)
df.to_csv('./datasets/PlaneAccidents.csv',index=False)

## Top100Fatalities

In [43]:
df2 = pd.read_html('http://www.planecrashinfo.com/worst100.htm',header=0)[0]

In [44]:
# drop index column
df2.drop(['Unnamed: 0','Photo'],axis=1,inplace=True)

In [45]:
#rename columns 
dicc = {
'Fatal': 'fatalities',                   
'Date': 'date',        
'Location': 'location',            
'Carrier': 'carrier',              
'Flight': 'flight',             
'Type': 'type'                  
}
df2 = df2.rename(columns=dicc)

In [46]:
# fix the 'Fatal'[0] as it had annotations from the web page
df2.fatalities[0] = 2907

In [47]:
# null values here are '-'
temp = df2.replace('-',None)
temp.isnull().sum()/len(temp) *100

fatalities    0.00000
date          0.00000
location      0.00000
carrier       0.00000
flight        9.90099
type          0.00000
dtype: float64

In [48]:
# get date in the same standard as PlaneAccidents.csv
df2.date = df2.date.apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))

In [49]:
#before saving it we need to change null values into real None types so we can get the whole column in the correct data type in MySQL
df2 = df2.replace('-',None)

In [50]:
# data is super clean and relevant to everything I'm gonna talk about so there's nothing else to do.
df2.to_csv('./datasets/Top100Fatalities.csv',index=False)